# Groundedness Checking Skills

In this notebook we run a simple grounding pipeline, to see if a summary text has any ungrounded additions as compared to 

Let us first define our grounding text:

In [1]:
grounding_text = """I am by birth a Genevese, and my family is one of the most distinguished of that republic.
My ancestors had been for many years counsellors and syndics, and my father had filled several public situations
with honour and reputation. He was respected by all who knew him for his integrity and indefatigable attention
to public business. He passed his younger days perpetually occupied by the affairs of his country; a variety
of circumstances had prevented his marrying early, nor was it until the decline of life that he became a husband
and the father of a family.

As the circumstances of his marriage illustrate his character, I cannot refrain from relating them. One of his
most intimate friends was a merchant who, from a flourishing state, fell, through numerous mischances, into poverty.
This man, whose name was Beaufort, was of a proud and unbending disposition and could not bear to live in poverty
and oblivion in the same country where he had formerly been distinguished for his rank and magnificence. Having
paid his debts, therefore, in the most honourable manner, he retreated with his daughter to the town of Lucerne,
where he lived unknown and in wretchedness. My father loved Beaufort with the truest friendship and was deeply
grieved by his retreat in these unfortunate circumstances. He bitterly deplored the false pride which led his friend
to a conduct so little worthy of the affection that united them. He lost no time in endeavouring to seek him out,
with the hope of persuading him to begin the world again through his credit and assistance.

Beaufort had taken effectual measures to conceal himself, and it was ten months before my father discovered his
abode. Overjoyed at this discovery, he hastened to the house, which was situated in a mean street near the Reuss.
But when he entered, misery and despair alone welcomed him. Beaufort had saved but a very small sum of money from
the wreck of his fortunes, but it was sufficient to provide him with sustenance for some months, and in the meantime
he hoped to procure some respectable employment in a merchant's house. The interval was, consequently, spent in
inaction; his grief only became more deep and rankling when he had leisure for reflection, and at length it took
so fast hold of his mind that at the end of three months he lay on a bed of sickness, incapable of any exertion.

His daughter attended him with the greatest tenderness, but she saw with despair that their little fund was
rapidly decreasing and that there was no other prospect of support. But Caroline Beaufort possessed a mind of an
uncommon mould, and her courage rose to support her in her adversity. She procured plain work; she plaited straw
and by various means contrived to earn a pittance scarcely sufficient to support life.

Several months passed in this manner. Her father grew worse; her time was more entirely occupied in attending him;
her means of subsistence decreased; and in the tenth month her father died in her arms, leaving her an orphan and
a beggar. This last blow overcame her, and she knelt by Beaufort's coffin weeping bitterly, when my father entered
the chamber. He came like a protecting spirit to the poor girl, who committed herself to his care; and after the
interment of his friend he conducted her to Geneva and placed her under the protection of a relation. Two years
after this event Caroline became his wife.
"""

And the summary we're going to examine:

In [2]:
summary_text = """The narrator's father was a respected public figure in Geneva. He was friends with a wealthy
merchant named Beaufort, who fell into poverty and retreated to Lucerne with his daughter. The narrator's father
searched for ten months before finding Beaufort, who was sick and unable to work. Beaufort died after several
months, leaving his daughter an orphan and a beggar. The narrator's father took her under his care and eventually
married her two years later.
"""

## Set up Semantic Kernel

We prepare our kernel in the usual way:

In [3]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))

## Import the Skill

We have to import the semantic and native skills in separate calls:

In [4]:
# note: using skills from the samples folder
skills_directory = "../../skills"

groundingSemanticFunctions = kernel.import_semantic_skill_from_directory(skills_directory, "GroundingSkill")
groundingNativeFunctions = kernel.import_native_skill_from_directory(skills_directory, "GroundingSkill")

In [ ]:
entity_extraction = groundingSemanticFunctions["EntityExtraction"]
reference_check = groundingSemanticFunctions["ReferenceCheck"]
jsonblock_fetch = groundingNativeFunctions["ExtractJsonBlock"]

## Preparing the context

We now create the context in which will will operate. In the beginning, this just specifies the kind of entities we want to check for grounding:

In [ ]:
context = kernel.create_new_context()
context["topic"] = "people and places"
context["example_entities"] = "names of people, towns and professions"

## Extracting the entities

We're now ready to get the entities from the summary text:

In [ ]:
extraction_result = entity_extraction(summary_text, context=context)

Examine the raw result

In [ ]:
print(extraction_result.result)

## Reference cross check

We now want to see if these entities appear in our grounding text. First, we place the grounding text into the general milieu of our kernel, so it can form background information:

In [ ]:
context["reference_context"] = grounding_text

With our grounding information in place, we can run the reference checking function (you didn't think that the pretty printing of the list of extracted entities was for _your_ benefit, did you?):

In [ ]:
grounding_result = reference_check(extraction_result.result, context=context)

print(grounding_result.result)

## Using a Planner

In [5]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [46]:
ask = f"""I have been given the following summary text:

<summary>{summary_text}</summary>

This text was based on the following original text:

<original>{grounding_text}</original>

I am interested in people (including those directly named,
those implied with familial relationships, and professions) and places.

I would like the list of people and places which are present in the summary text but which are NOT present in the original text.
"""

print(ask)

I have been given the following summary text:

<summary>The narrator's father was a respected public figure in Geneva. He was friends with a wealthy
merchant named Beaufort, who fell into poverty and retreated to Lucerne with his daughter. The narrator's father
searched for ten months before finding Beaufort, who was sick and unable to work. Beaufort died after several
months, leaving his daughter an orphan and a beggar. The narrator's father took her under his care and eventually
married her two years later.
</summary>

This text was based on the following original text:

<original>I am by birth a Genevese, and my family is one of the most distinguished of that republic.
My ancestors had been for many years counsellors and syndics, and my father had filled several public situations
with honour and reputation. He was respected by all who knew him for his integrity and indefatigable attention
to public business. He passed his younger days perpetually occupied by the affairs of his count

In [47]:
my_plan = await planner.create_plan_async(ask, kernel)

In [48]:
print(my_plan.generated_plan)

{
    "input": "<original>I am by birth a Genevese, and my family is one of the most distinguished of that republic. My ancestors had been for many years counsellors and syndics, and my father had filled several public situations with honour and reputation. He was respected by all who knew him for his integrity and indefatigable attention to public business. He passed his younger days perpetually occupied by the affairs of his country; a variety of circumstances had prevented his marrying early, nor was it until the decline of life that he became a husband and the father of a family.\n\nAs the circumstances of his marriage illustrate his character, I cannot refrain from relating them. One of his most intimate friends was a merchant who, from a flourishing state, fell, through numerous mischances, into poverty. This man, whose name was Beaufort, was of a proud and unbending disposition and could not bear to live in poverty and oblivion in the same country where he had formerly been dist

In [49]:
results = await planner.execute_plan_async(my_plan, kernel)

In [50]:
print(results)


- Genevese
- Counsellors
- Syndics
- Public situations
- Beaufort
- Reuss
- Merchant's house
- Caroline Beaufort


In [23]:
print(my_plan.prompt)


You are a planner for the Semantic Kernel.
Your job is to create a properly formatted JSON plan step by step, to satisfy the goal given.
Create a list of subtasks based off the [GOAL] provided.
Each subtask must be from within the [AVAILABLE FUNCTIONS] list. Do not use any functions that are not in the list.
Base your decisions on which functions to use from the description and the name of the function.
Sometimes, a function may take arguments. Provide them if necessary.
The plan should be as short as possible.
For example:

[AVAILABLE FUNCTIONS]
EmailConnector.LookupContactEmail
description: looks up the a contact and retrieves their email address
args:
- name: the name to look up

WriterSkill.EmailTo
description: email the input text to a recipient
args:
- input: the text to email
- recipient: the recipient's email address. Multiple addresses may be included if separated by ';'.

WriterSkill.Translate
description: translate the input to another language
args:
- input: the text to tr